In [23]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVR
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import cross_val_score

In [93]:
#load in the 100k feather file
df = pd.read_feather('100k_kindle_reviews.feather')

In [16]:
df.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,processed_review
0,1,True,"12 27, 2014",A3JRP9RI3ZE6VK,B00JQDHIES,{'Format:': ' Kindle Edition'},e-BOOK,Truly hate it when I make a bad call on a book...,Juvenile,1419638400,0,None,truly hate bad book waste money . honestly fin...
1,1,False,"06 25, 2016",A3IZWB0YUZXZZ6,B01H124NQQ,{'Format:': ' Kindle Edition'},Kathryn childs,Only 4% of this book is the discretion it isn'...,Not worth your time,1466812800,0,None,4 % book discretion worth time money . say 800...
2,1,True,"05 13, 2013",A82LP6E48UFM5,B00C2E9EK4,{'Format:': ' Kindle Edition'},Amazon Customer,The quality of writing for this book is very p...,poor quality,1368403200,0,None,quality writing book poor . understand insta...
3,1,True,"05 10, 2012",AMV9B31WKSLM7,B005RH2GNU,{'Format:': ' Kindle Edition'},Eclectic Reader,"The Outback is the vast, remote, arid area of ...",Outback is in AUSTRALIA!!!,1336608000,0,None,"Outback vast , remote , arid area Australia ac..."
4,1,True,"09 4, 2013",AZ27B4OUL64TR,B00DC7PXLY,{'Format:': ' Kindle Edition'},Word Junkie,"Grace Burrowes lives within a few miles of me,...",Not as good as its progenitor,1378252800,0,None,"Grace Burrowes live mile , want tell enjoy Gro..."


In [11]:
# Load the Spacy language model
nlp = spacy.load("en_core_web_sm")
nlp.disable_pipes(['parser', 'ner'])
print(nlp.pipe_names)

['tok2vec', 'tagger', 'attribute_ruler', 'lemmatizer']


In [45]:
# Function to perform lemmatization and stop word removal
def preprocess_text(text):
    doc = nlp(text)
    lemmatized_text = ' '.join([token.lemma_ for token in doc if not token.is_stop])
    # create a dictionary for the different word types in pos_ incrementing everytime a token is found of that type
    dict_of_types = {}
    # Iterate over the tokens in the document
    for token in doc:
        if token.pos_ in dict_of_types:
            dict_of_types[token.pos_] += 1
        else:
            dict_of_types[token.pos_] = 1

    return lemmatized_text, dict_of_types

In [ ]:
result_df = pd.DataFrame(columns=['lemmatized_reviewText'])
text_list = df['reviewText'].tolist()
for text in text_list:
    lemmatized_text, dict_of_types = preprocess_text(text)
    result_df = result_df.append({'lemmatized_reviewText': lemmatized_text, **dict_of_types}, ignore_index=True)

In [47]:
#for all of the columns if not named lemmatized_reviewText rename it with a _review
for col in result_df.columns:
    if col != 'lemmatized_reviewText':
        result_df = result_df.rename(columns={col: col + '_review'})

In [76]:
text_list = df['summary'].tolist()
result_df2 = pd.DataFrame(columns=['lemmatized_summary'])
for text in text_list:
    lemmatized_text, dict_of_types = preprocess_text(text)
    result_df2 = result_df2.append({'lemmatized_summary': lemmatized_text, **dict_of_types}, ignore_index=True)

Streaming output truncated to the last 5000 lines.
<ipython-input-76-c3dd2297e073>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df2 = result_df2.append({'lemmatized_summary': lemmatized_text, **dict_of_types}, ignore_index=True)
<ipython-input-76-c3dd2297e073>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df2 = result_df2.append({'lemmatized_summary': lemmatized_text, **dict_of_types}, ignore_index=True)
<ipython-input-76-c3dd2297e073>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df2 = result_df2.append({'lemmatized_summary': lemmatized_text, **dict_of_types}, ignore_index=True)
<ipython-input-76-c3dd2297e073>:7: FutureWarning: The frame.append method is deprecated and will be removed from pa

In [77]:
# rename the columns
for col in result_df2.columns:
    if col != 'lemmatized_summary':
        result_df2 = result_df2.rename(columns={col: col + '_summary'})

In [78]:
# merge results and results2
result_df = result_df.merge(result_df2, left_index=True, right_index=True)

In [87]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 39 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   lemmatized_reviewText  100000 non-null  object 
 1   ADV_review             80797 non-null   float64
 2   VERB_review            91419 non-null   float64
 3   PRON_review            86519 non-null   float64
 4   SCONJ_review           47676 non-null   float64
 5   DET_review             89456 non-null   float64
 6   ADJ_review             92077 non-null   float64
 7   NOUN_review            85753 non-null   float64
 8   ADP_review             77298 non-null   float64
 9   CCONJ_review           66467 non-null   float64
 10  PUNCT_review           83850 non-null   float64
 11  AUX_review             76371 non-null   float64
 12  PART_review            64968 non-null   float64
 13  PROPN_review           46241 non-null   float64
 14  NUM_review             30256 non-null

In [90]:
result_df.shape

(100000, 39)

In [94]:
# append the results dataframe onto the orignal df and then make a feather file
df = df.join(result_df)
df.to_feather('100k_kindle_reviews_lemmatized_with_summary.feather')

In [95]:
vectorizer = CountVectorizer()

# Fit CountVectorizer to learn the vocabulary (word dictionary)
vectorizer.fit(result_df['lemmatized_reviewText'])

CountVectorizer()

In [ ]:
X = vectorizer.transform(result_df['lemmatized_reviewText'])

In [ ]:
svd = TruncatedSVD(n_components=min(X.shape) - 1)  # Set maximum number of components

# Fit TruncatedSVD on the sparse matrix
svd.fit(X)

# Plotting the explained variance ratio
plt.figure(figsize=(8, 6))
plt.plot(range(1, svd.n_components + 1), svd.explained_variance_ratio_.cumsum(), marker='o', linestyle='--')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Explained Variance Ratio vs. Number of Components')
plt.grid(True)
plt.show()